## Improt

In [ ]:
import pandas as pd

df = pd.read_csv("store.csv")

## Cleaning

In [ ]:
# Nettoyage des donnes
df["Order Date"] = pd.to_datetime(df["Order Date"],format="%d/%m/%Y", errors="coerce")
df["Ship Date"] = pd.to_datetime(df["Ship Date"],format="%d/%m/%Y", errors="coerce")
df.dropna(subset=["Postal Code"], inplace=True)
df.drop_duplicates(inplace=True)
# Hypothèse: cost = 60% of sales
df["Cost_product"] = df["Sales"] * 0.6

## Feature Engineering

In [ ]:
df["Years"] = df["Order Date"].dt.year
df["Trimester"] = df["Order Date"].dt.quarter
df["Months"] = df["Order Date"].dt.month
# Profit total
df["Profit"] = df["Sales"] - df["Cost_product"]
# Profit ratio
df["Profit_ratio"] = df["Profit"] / df["Sales"] * 100
# délais livraison
df["Delivery_Delay"] = (df["Ship Date"] - df["Order Date"]).dt.days
max_min_delais_livraison = df["Delivery_Delay"].agg(["min", "max"])
delais_livraison_region = df.groupby("Region")["Delivery_Delay"].mean()

## KPIs Commercial

In [ ]:
## Direction Commerciale

# Chiffre d’affaires total
CA_total = df["Sales"].sum()

# Marge moyenne
Marge_profit = df["Profit_ratio"].mean()

# Ventes par catégorie
Ventes_categorie = df.groupby("Category")["Sales"].sum().sort_values(ascending=False)

# Ventes par région
Ventes_region = df.groupby("Region")["Sales"].sum().sort_values(ascending=False)

# Top 10 produits
Top_10_products = df.groupby("Product Name")["Sales"].sum().sort_values(ascending=False).head(10)

# Croissance mensuelle des ventes
Croissance_mensuelle_ventes = df.groupby(["Years", "Months"])["Sales"].sum().sort_index().pct_change() * 100

## KPIs Finance

In [ ]:
## Direction financier

# Profit moyenne
Profit_moyenne = df["Profit"].mean()

# Profit minimum / maximum
Profit_min_max = df["Profit"].agg(["min", "max"])

# Écart-type des profits
Ecart_type = df["Profit"].std()

# Produits / catégories à faible marge
P_F_marge = df.groupby("Product Name")["Profit_ratio"].mean().sort_values().head()
C_F_marge = df.groupby("Category")["Profit_ratio"].mean().sort_values()

# Outliers
Q1 = df["Profit"].quantile(0.25)
Q3 = df["Profit"].quantile(0.75)
IQR = Q3 - Q1
borne_inf = Q1 - 1.5 * IQR
borne_sup = Q3 + 1.5 * IQR
Outliers = df[(df["Profit"] < borne_inf) | (df["Profit"] > borne_sup)]

## Validation

In [ ]:
df.info()
df.describe()
df.duplicated().sum()
df.isna().sum()
df.dtypes
df.shape

## Exporting

In [ ]:
df.to_csv("superstore_clean.csv", index=False)